In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [2]:
# Load the data
merged_data = pd.read_csv('data/merged_data.csv')

# convert 'ts' column to datetime format
data_opti = merged_data.copy()
data_opti['ts'] = pd.to_datetime(merged_data['ts'])

# create a column with a day counter, knowing that ts is in the format 2023-01-02 00:00:00
data_opti['day_counter'] = (data_opti['ts'] - data_opti['ts'][0]).dt.days

# filter merged_data to have only the days higher than day_init and the days lower than day_end
day_init = 6
day_end = 7
data_opti = data_opti[(data_opti['day_counter'] >= day_init) & (data_opti['day_counter'] <= day_end)]

In [3]:
class Expando(object):
    '''
        A small class which can have attributes set
    '''
    pass


class InputData:
    
    def __init__(
        self, 
        wind_forecast: dict[str, float],
        data_opti: pd.DataFrame,
    ):
        self.hours = [str(i) for i in range(24)]
        self.wind_forecast = wind_forecast
        self.wind_realization = {str(i): data_opti['Hasle Vind Active Power | has_vin_effekt | 804123'].iloc[i]  for i in range(24)}
        self.price_da = {str(i): data_opti['Nordpool Elspot Prices - hourly price DK-DK2 EUR/MWh | 9F7J/00/00/Nordpool/DK2/hourly_spot_eur | 3038'].iloc[i]  for i in range(24)}
        self.price_bal_up = {str(i): data_opti['BalancingPowerPriceDownEUR | BalancingPowerPriceDownEUR | 804720'].iloc[i]  for i in range(24)}
        self.price_bal_down = {str(i): data_opti['BalancingPowerPriceUpEUR | BalancingPowerPriceUpEUR | 804718'].iloc[i]  for i in range(24)}
        self.nameplate_capacity = 100


class Step_6_Optimized_Bid():

    def __init__(self, input_data: InputData):
        self.data = input_data 
        self.variables = Expando()
        self.constraints = Expando() 
        self.results = Expando() 
        self._build_model() 

    def _build_variables(self):
        self.variables.bid_DA = {
            h: self.model.addVar(
                lb=0, ub=GRB.INFINITY, name='Bid in day-ahead market'
            ) for h in self.data.hours
            }
        
        self.variables.delta = {
            h: self.model.addVar(
                lb=-GRB.INFINITY, ub=GRB.INFINITY, name='Difference between bid and realization'
            ) for h in self.data.hours
            }

        self.variables.delta_up = {
            h: self.model.addVar(
                lb=0, ub=GRB.INFINITY, name='Positive difference between bid and realization'
            ) for h in self.data.hours
            }
        
        self.variables.delta_down = {
            h: self.model.addVar(
                lb=0, ub=GRB.INFINITY, name='Negative difference between bid and realization'
            ) for h in self.data.hours
            }
        
    
    def _build_constraints(self):
        self.constraints.bid_DA = {
            h: self.model.addConstr(
                self.variables.bid_DA[h] <= self.data.nameplate_capacity, 
                name='Bid in day-ahead market should be less than or equal to nameplate capacity'
            ) for h in self.data.hours
            }
        
        self.constraints.delta = {
            h: self.model.addConstr(
                self.variables.delta[h] == self.variables.bid_DA[h] - self.data.wind_forecast[h], 
                name='Difference between bid and forecast'
            ) for h in self.data.hours
            }

        self.constraints.delta_up = {
            h: self.model.addConstr(
                self.variables.delta[h] == self.variables.delta_up[h] - self.variables.delta_down[h],
            ) for h in self.data.hours
            }
        


    def _build_objective_function(self):
        
        objective = (
            gp.quicksum(
                self.data.price_da[h] * self.variables.bid_DA[h] + 
                self.data.price_bal_up[h] * self.variables.delta_up[h] - 
                self.data.price_bal_down[h] * self.variables.delta_down[h]
                for h in self.data.hours
            )
        )
        self.model.setObjective(objective, GRB.MAXIMIZE)

    def _build_model(self):
        self.model = gp.Model(name='Optimized DA bid for 24 hours')
        self._build_variables()
        self._build_constraints()
        self._build_objective_function()
        self.model.update()

    def _save_results(self):
        self.results.bid_DA = {
            h: self.variables.bid_DA[h].x for h in self.data.hours
        }

        self.results.wind_forecast = {
            h: self.data.wind_forecast[h] for h in self.data.hours
        }
        self.results.forecasted_delta = {
            h: self.variables.delta[h].x for h in self.data.hours
        }
        self.results.forecasted_profit_DA = {
            h: self.data.price_da[h] * self.variables.bid_DA[h].x for h in self.data.hours
        }
        self.results.forecasted_profit_bal = {
            h: self.data.price_bal_up[h] * self.variables.delta_up[h].x - self.data.price_bal_down[h] * self.variables.delta_down[h].x for h in self.data.hours
        }
        self.results.total_forecasted_profit = {
            h: self.results.forecasted_profit_DA[h] + self.results.forecasted_profit_bal[h] for h in self.data.hours
        }
        
        self.results.realization = {
            h: self.data.wind_realization[h] for h in self.data.hours
        }
        self.results.realised_delta = {
            h: self.results.bid_DA[h] - self.results.realization[h] for h in self.data.hours
        }
        self.results.realised_profit_DA = {
            h: self.data.price_da[h] * self.results.realization[h] for h in self.data.hours
        }
        self.results.realised_profit_bal = {
            h: self.data.price_bal_up[h] * self.results.realised_delta[h] if self.results.realised_delta[h] > 0 else self.data.price_bal_down[h] * self.results.realised_delta[h] for h in self.data.hours
        }
        self.results.total_realised_profit = {
            h: self.results.realised_profit_DA[h] + self.results.realised_profit_bal[h] for h in self.data.hours
        }

        results = {
            'Hour': list(self.data.hours),
            'DA bid': list(self.results.bid_DA.values()),

            'Forecasted wind': list(self.results.wind_forecast.values()),
            'Forecasted delta': list(self.results.forecasted_delta.values()),
            'Forecasted profit DA': list(self.results.forecasted_profit_DA.values()),
            'Forecasted profit Bal': list(self.results.forecasted_profit_bal.values()),
            'Total forecasted profit': list(self.results.total_forecasted_profit.values()),

            'Realization': list(self.results.realization.values()),
            'Realised delta': list(self.results.realised_delta.values()),
            'Realised profit DA': list(self.results.realised_profit_DA.values()),
            'Realised profit Bal': list(self.results.realised_profit_bal.values()),
            'Total realised profit': list(self.results.total_realised_profit.values()),
        }

        # Create a DataFrame using this reorganized data
        self.results.df = pd.DataFrame(results)

        # Create a result attribute with the total profit of the day
        self.results.total_realised_profit_day = sum(self.results.total_realised_profit.values())

        


    def run(self):
        self.model.optimize()
        if self.model.status == GRB.OPTIMAL:
            self._save_results()
        else:
            raise RuntimeError(f"optimization was not successful")
    
    def display_results(self):
        print()
        print("-------------------   RESULTS  -------------------")
        print("Detailled results:")
        print(self.results.df)
        print("--------------------------------------------------")

    
   


In [4]:
# create an input data object with random data for 24 hours
input_data = InputData(
    wind_forecast = {str(i): 100 for i in range(24)},
    data_opti = data_opti
)
    

In [5]:
model = Step_6_Optimized_Bid(input_data)
model.run()
model.display_results()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-24
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 72 rows, 96 columns and 144 nonzeros
Model fingerprint: 0xe5daa000
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+02]
Presolve removed 72 rows and 96 columns
Presolve time: 0.03s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1215900e+05   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.04 seconds (0.00 work units)
Optimal objective  1.121590000e+05

-------------------   RESULTS  -------------------
Detailled results:
   Hour  DA bid  Forecasted wind  Forecasted delta 